<a href="https://colab.research.google.com/github/loureenvalyne/R_projects/blob/main/nursery_survey_data_Analyis_with_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATA_SOURCE_MAPPING = 'nursery-survey-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F6092259%2F9914335%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241117%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241117T011939Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D944fc9b26c5045e63920c686db9f6b58637a5e676f3add91d7a9bcf2d75ff71df99de01242016ba39e1e1228afc1d93978b839f662653b06e21d82178446576536387b8ee99cc90264a9bc5c2c002c2ca6704f5a4a1abf278d607935d3c5036e23c02c1b1d67da5a102dc42c53e1a668d4699dad4e9ced46e5e2622e1a1bf8169240cb435c5d88b79ee03cd02c4c9a27cc519c88b40a60839f865281952c8e63eed6981481f4119385e42b7ad5c340a9118b461950ad15e13a9a004b5dd821cab4460fbb6215c6d6b9584081763b8895edefb1314e43e204af768a91ec28ec87148b315a5e576f4b98473bbaa4aaaab3a29dd9e760274f427c076105ab72bec0'

KAGGLE_INPUT_PATH = '/kaggle/input'
KAGGLE_WORKING_PATH = '/kaggle/working'

system(paste0('sudo umount ', '/kaggle/input'))
system(paste0('sudo rmdir ', '/kaggle/input'))
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH,' ',file.path('..', 'input')),
  intern=TRUE)

system(paste0('sudo mkdir -p -- ', KAGGLE_WORKING_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_WORKING_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_WORKING_PATH, ' ', file.path('..', 'working')),
  intern=TRUE)

data_source_mappings = strsplit(DATA_SOURCE_MAPPING, ',')[[1]]
for (data_source_mapping in data_source_mappings) {
    path_and_url = strsplit(data_source_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    filename = sub("\\?.+", "", download_url)
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    print(paste0('Downloading and uncompressing: ', directory))
    if (endsWith(filename, '.zip')){
      temp = tempfile(fileext = '.zip')
      download.file(download_url, temp)
      unzip(temp, overwrite = TRUE, exdir = destination_path)
      unlink(temp)
    }
    else{
      temp = tempfile(fileext = '.tar')
      download.file(download_url, temp)
      untar(temp, exdir = destination_path)
      unlink(temp)
    }
    print(paste0('Downloaded and uncompressed: ', directory))
}

print(paste0('Data source import complete'))


# NURSERY SURVEY DATA - DATA ANALYSIS
Tree nurseries are integral to afforestation and reforestation initiatives as they provide the seedlings needed for successful restoration. This analysis focuses on survey data collected from various tree nurseries to assess their capacity to produce sufficient seedlings for ecological restoration projects.

The primary objective of this analysis is to understand the production capacity of nurseries across different counties. By examining variables such as the types of seedlings produced, nursery capacity, and other key indicators, we aim to identify regional strengths and gaps in the nurseries’ ability to meet restoration demands. Through the use of pivot tables, charts, and summary statistics, this analysis provides insights into the resources and capabilities available at each nursery and highlights areas for potential support or development.

## Installing packages for Analysis

In [ ]:
install.packages("tidyverse")
library(tidyverse)
library(lubridate)
library(dplyr)
library(ggplot2)
library(tidyr)
library(readr)

## Importing the Data-set

In [ ]:
nursery_survey_data <- read_csv("/kaggle/input/nursery-survey-data/nursery_survey_data.csv")

## Exploring the Data-set

In [ ]:
# checking the outlook of the data, the first 10 rows
head(nursery_survey_data, 10)

In [ ]:
# checking the attributes of the data
colnames(nursery_survey_data)

In [ ]:
#checking for missing values and NA's
summary(nursery_survey_data)

# Pivot Table for Organization, Respondent Status, and County

We will create a pivot table that summarizes the counts of respondent status (owner/employee) categorized by county (ADM1) and organization.

In [ ]:
pivot_table_status <- nursery_survey_data %>%
  group_by(organization,ADM1,respondent_status) %>%
  summarise(
    count = n_distinct(respondent_name, na.rm = TRUE),  # Count unique respondents
    .groups = "drop"
  ) %>%
  pivot_wider(
    names_from = respondent_status,
    values_from = count,
    values_fill = list(count = 0)  # Fill missing values with 0
  )

# View the pivot table
View(pivot_table_status)

### Creating a Unique Identifier for Nurseries

In [ ]:
nursery_survey_data <- nursery_survey_data %>%
  mutate(nursery_id = dense_rank(respondent_name))


# RESPONDENTS ANALYSIS
## Total number of respondents

In [ ]:
pivot_total_respondents <- nursery_survey_data %>%
  group_by(ADM1) %>%  # Group by county (ADM1)
  summarise(
    total_respondents = n_distinct(respondent_name, na.rm = TRUE),  # Count unique respondents
    .groups = "drop"
  )

# Add a 'Total' row that sums the 'total_respondents' column
pivot_total_respondents <- pivot_total_respondents %>%
  add_row(ADM1 = "Total", total_respondents = sum(pivot_total_respondents$total_respondents))

# View the resulting pivot table with the 'Total' row
print(pivot_total_respondents)

## Getting the Mean for the Gender in The Number of Respondents

In [ ]:
gender_stats_per_county <- nursery_survey_data %>%
  group_by(ADM1) %>%
  summarise(
    count_male = n_distinct(respondent_name[gender == "male"], na.rm = TRUE),   # Unique count of males
    count_female = n_distinct(respondent_name[gender == "female"], na.rm = TRUE), # Unique count of females
    total_count = count_male + count_female,                                  # Total unique count
    mean_male = round((count_male / total_count) * 100, 2),                   # Mean percentage of males
    mean_female = round((count_female / total_count) * 100, 2),               # Mean percentage of females
    .groups = 'drop'
  )

# View the result
print(gender_stats_per_county)

## Nursery Experience in Years for each respondent

In [ ]:
# Function to calculate the mode of a vector
get_mode <- function(x) {
  uniqx <- unique(x)
  uniqx[which.max(tabulate(match(x, uniqx)))]
}

# Create the pivot table and get the mode of years of experience per respondent
pivot_table_experience <- nursery_survey_data %>%
  select(respondent_name, ADM1, respondent_status, nursery_experience_years) %>%
  group_by(respondent_name, ADM1,respondent_status) %>%
  summarise(
    total_nursery_experience_years = get_mode(nursery_experience_years),  # Get the mode of experience years
    .groups = 'drop'
  )

# View the result
View(pivot_table_experience)

# NURSERY ANALYSIS
## Summarizing the Nursery Capacity by County named as ADM1
To understand the capacity by county we group and summarize the data by County. This will give us insights into total and average seedling capacity in each county.To analyze the total and average seedling production capacity by county, we use group_by() and summarize() functions. Assume number_of_seedlings represents the capacity of each nursery.

In [ ]:
# Convert `number_of_seedlings` to numeric, handling any non-numeric values as NA
nursery_survey_data$number_of_seedlings <- as.numeric(nursery_survey_data$number_of_seedlings)

county_summary <- nursery_survey_data %>%
  group_by(ADM1) %>%
  summarize(
    total_seedlings = sum(number_of_seedlings, na.rm = TRUE),
    average_seedlings = mean(number_of_seedlings, na.rm = TRUE),
    unique_nurseries_count_per_county = n_distinct(nursery_id)  # Count unique nurseries
  )

# View summary
print(county_summary)

## Analyzing the nursery types and nursery purpose in each county
## Nursery Type Per County

In [ ]:
pivot_table_nursery <- nursery_survey_data %>%
  group_by(ADM1, nursery_type) %>%
  summarize(count = n_distinct(nursery_id), .groups = "drop")  # Count unique nurseries

custom_colors <- c(
  "#4E79A7", "#59A14F", "#F28E2B", "#E15759", "#76B7B2",
  "#FF9DA7", "#9C755F", "#B07AA1", "#F1C1D1", "#92C5DA",
  "#B7C8F5"
)

ggplot(pivot_table_nursery, aes(x = ADM1, y = count, fill = ADM1)) +
  geom_bar(stat = "identity", position = "dodge") +
  facet_wrap(~nursery_type) +  # Create separate plots for each nursery type
  scale_fill_manual(values = custom_colors) +  # Manually specify colors
  labs(
    title = "Nursery Type by County",
    x = "County",
    y = "Unique Nursery Count",  # Updated y-axis label
    fill = "County"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    legend.position = "none"  # Remove the legend
  ) +
  # Add count labels on top of bars
  geom_text(aes(label = count),
            vjust = -0.3,  # Adjust vertical position of the labels
            position = position_dodge(width = 0.9),  # Align text with bars
            size = 3)  # Adjust text size

## Count of Unique Nurseris per County

In [ ]:
# Summarize data by counting unique nurseries (nursery_id) per county
pivot_table_nursery <- nursery_survey_data %>%
  group_by(ADM1) %>%
  summarize(count = n_distinct(nursery_id), .groups = "drop")  # Count unique nursery_id

# Custom colors for the plot
custom_colors <- c(
  "#4E79A7", "#59A14F", "#F28E2B", "#E15759", "#76B7B2",
  "#FF9DA7", "#9C755F", "#B07AA1", "#F1C1D1", "#92C5DA",
  "#B7C8F5"
)

# Create the bar chart
ggplot(pivot_table_nursery, aes(x = ADM1, y = count, fill = ADM1)) +
  geom_bar(stat = "identity", position = "dodge", width = 0.8) +
  geom_text(
    aes(label = count),
    vjust = -0.5,  # Position the text slightly above the bar
    size = 3.5
  ) +
  scale_fill_manual(values = custom_colors) +  # Manually specify colors
  labs(
    title = "Unique Nurseries by County",
    x = "County",
    y = "Unique Nursery Count",
    fill = "County"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    legend.position = "none"  # Remove the legend
  )

Different Counties have more than one unique nurseries

## Nursery Purpose
## Pivot tables for grouping the Nurseries by Purpose for each counties

In [ ]:
pivot_table_purpose <- nursery_survey_data %>%
  group_by(ADM1, nursery_purpose) %>%
  summarize(count = n(), .groups = "drop")  # Count the occurrences of each nursery purpose

# Pivot the data to make it wider (create a column for each nursery_purpose)
pivot_table_wide_purpose <- pivot_table_purpose %>%
  pivot_wider(
    names_from = nursery_purpose,  # Creates columns for each nursery_purpose
    values_from = count,           # The count of each purpose in each county
    values_fill = list(count = 0)  # Fill missing values with 0
  )

# View the pivot table
print(pivot_table_wide_purpose)

In [ ]:
# Summarize data by counting unique nurseries (nursery_id) for each nursery purpose
purpose_count <- nursery_survey_data %>%
  group_by(nursery_purpose) %>%
  summarize(count = n_distinct(nursery_id), .groups = "drop")  # Count unique nursery_id

# Reorder nursery_purpose based on the count, from most to least frequent
purpose_count <- purpose_count %>%
  mutate(nursery_purpose = fct_reorder(nursery_purpose, count))

# Create a bar graph with ggplot2
ggplot(purpose_count, aes(x = nursery_purpose, y = count, fill = nursery_purpose)) +
  geom_bar(stat = "identity") +  # Create a bar graph
  geom_text(
    aes(label = count),  # Add the count on top of each bar
    vjust = -0.5,  # Position text slightly above the bar
    color = "black",
    size = 3.5
  ) +
  scale_fill_brewer(palette = "Set3") +  # Use a nice color palette
  labs(
    title = "Unique Nurseries by Purpose",
    x = "Nursery Purpose",
    y = "Unique Nursery Count"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1, color = "black"),  # Maintain x-axis text
    axis.text.y = element_text(color = "black"),  # Black text for y-axis
    axis.title.y = element_text(color = "black"),  # Black title for y-axis
    legend.position = "none"  # Remove the legend
  )

From the analysis we can see that majority of the nurseries produce seeds for income forest management in-house needs while the least produce for income forest management in general not just their own in-house needs

# SEEDLINGS ANALYSIS(INDIGENOUS & EXOTIC)
## Finding the types of seedlings per county

In [ ]:
# Summarize the total counts of Indigenous and Exotic seedlings by county
pivot_table <- nursery_survey_data %>%
  group_by(ADM1) %>%
  summarize(
    count_indigenous = sum(Indigenous, na.rm = TRUE),
    count_exotic = sum(Exotic, na.rm = TRUE),
    total_count = count_indigenous + count_exotic,  # Calculate total count
    mean_indigenous = round((count_indigenous / total_count) * 100, 2),  # Percentage for Indigenous
    mean_exotic = round((count_exotic / total_count) * 100, 2),          # Percentage for Exotic
    .groups = "drop"  # Avoid nesting the results
  )

# View the pivot table
print(pivot_table)

## Distribution of Indigeous and Exotic Across the Counties

In [ ]:
pivot_table <- pivot_table %>%
  mutate(total_count = count_indigenous + count_exotic) %>%  # Calculate total seedlings per county
  arrange(desc(total_count)) %>%  # Arrange in descending order
  mutate(ADM1 = factor(ADM1, levels = unique(ADM1)))  # Reorder ADM1 based on total_count

# Plot with reordered ADM1
ggplot(pivot_table, aes(x = ADM1)) +
  geom_bar(aes(y = count_indigenous, fill = "Indigenous"), stat = "identity", position = "dodge") +
  geom_bar(aes(y = count_exotic, fill = "Exotic"), stat = "identity", position = "dodge") +
  labs(
    title = "Seedling Counts Across Counties",
    x = "County (ADM1)",
    y = "Count of Seedlings",
    fill = "Seedling Type"
  ) +
  scale_fill_manual(values = c("Indigenous" = "lightblue", "Exotic" = "blue")) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1, color = "black"),
         axis.text.y = element_text(color = "black"),  # Black text for y-axis
    axis.title.x = element_text(color = "black"),  # Black title for x-axis
    axis.title.y = element_text(color = "black")   # Black title for y-axis)
  )

# NURSERY INFRASTUCTURE AND MATERIALS
## Nursery Infrasture Appropriateness Per County

In [ ]:
# Data preparation: Count nursery_infrastructure_appropriateness responses per county
infrastructure_data <- nursery_survey_data %>%
  filter(!is.na(Nursery_Infrastructure_Appropriateness)) %>%  # Remove NA values
  group_by(ADM1, Nursery_Infrastructure_Appropriateness) %>%
  summarize(count = n(), .groups = "drop")  # Count responses for each county and appropriateness status

# Define custom shades of blue for each infrastructure appropriateness level
blue_shades <- c("poor" = "#F1C1D1", "fair" = "#E15759", "good" = "#4169e1", "very_good" = "#00008B")

# Create the stacked bar chart
ggplot(infrastructure_data, aes(x = ADM1, y = count, fill = Nursery_Infrastructure_Appropriateness)) +
  geom_bar(stat = "identity") +  # Default stacking for bars
  labs(
    title = "Nursery Infrastructure Appropriateness per County",
    x = "County",
    y = "Count",
    fill = "Infrastructure Appropriateness"
  ) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1, color = "black")) +  # Rotate x-axis labels for readability
  scale_fill_manual(values = blue_shades)  # Apply custom shades of blue

From the graph we discover that Kiambu County has the most very_good infrastructure appropriateness followed by Nakuru. Nakuru has a ratio of poor infrastructure whereas Kajiado and West-Pokot have complete fair and poor infrastructure. Laikipia, Meru and Narok have good infrastructures. The rest that is Baringo, Elgeyo Marakwet, Makueni and Nyandarua have a share of both good and fair infrastructures. Kiambu has a mix of both very good and fair while Nakuru seems to have signifacant ratios of all 4 categories

# Materials Satisfaction Per County

In [ ]:
# Data preparation: Calculate percentage of responses per satisfaction level within each county
satisfaction_data <- nursery_survey_data %>%
  filter(!is.na(Nursery_Materials_Satisfaction)) %>%  # Remove NA values
  group_by(ADM1, Nursery_Materials_Satisfaction) %>%
  summarize(count = n(), .groups = "drop") %>%  # Count responses for each county and satisfaction level
  group_by(ADM1) %>%
  mutate(percentage = count / sum(count) * 100)  # Calculate percentage per county

# Define custom shades of green for each satisfaction level
green_shades <- c("not_satisfied" = "#808080",
                  "somehow_satisfied" = "#a1d99b",
                  "satisfied" = "#32CD32",
                  "very_satisfied" = "#006d2c")

# Create the stacked bar chart
ggplot(satisfaction_data, aes(x = ADM1, y = percentage, fill = Nursery_Materials_Satisfaction)) +
  geom_bar(stat = "identity") +  # Default stacking for bars
  geom_text(
    aes(label = paste0(round(percentage, 1), "%")),  # Format percentages with 1 decimal point
    position = position_stack(vjust = 0.5),  # Center labels within each bar segment
    size = 3, color = "black"  # black text for better visibility
  ) +
  labs(
    title = "Nursery Materials Satisfaction per County",
    x = "County",
    y = "Percentage",
    fill = "Satisfaction Level"
  ) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1, color = "black")) +  # Rotate x-axis labels for readability
  scale_fill_manual(values = green_shades)  # Apply custom shades of green

From the bars, we can identify that Kiambu has a ratio of all satisfactions but the majority are unsatisfied with the nursery materials present. For the rest of the counties there is some level of satisfied apart from Kajiado, Narok and Nyandarua that incline more to somehow satisfied and not satisfied

# Finding the registration status and registering Organizations for each County
## Registration Status

In [ ]:
# Summarize and calculate proportions for registration status counts per county
pivot_table_reg_status <- nursery_survey_data %>%
  group_by(ADM1, nursery_registration_status) %>%
  summarise(count = n(), .groups = "drop") %>%
  complete(ADM1, nursery_registration_status, fill = list(count = 0)) %>%  # Fill missing with 0
  group_by(ADM1) %>%
  mutate(
    proportion = count / sum(count),  # Calculate proportions
    percentage = proportion * 100     # Calculate percentage for labeling
  )

# Create the 100% stacked bar chart with percentage labels, filtering out near-0% values
ggplot(pivot_table_reg_status, aes(x = ADM1, y = proportion, fill = nursery_registration_status)) +
  geom_bar(stat = "identity", position = "fill") +  # 100% stacked bar chart
  geom_text(
    aes(label = ifelse(percentage >= 1, paste0(round(percentage, 1), "%"), "")),  # Show labels for values >= 1%
    position = position_fill(vjust = 0.5), size = 3
  ) +
  labs(
    title = "Registration Status Distribution per County (100% Stacked with Percentages)",
    x = "County",
    y = "Proportion",
    fill = "Registration Status"
  ) +
  theme_minimal() +
  scale_fill_manual(values = c("skyblue", "lightgreen", "coral", "purple", "orange")) +  # Define colors
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1, color="black"),  # Rotate county labels for readability
    plot.title = element_text(hjust = 0.5),
    axis.text.y = element_text(color = "black"),  # Black text for y-axis
    axis.title.x = element_text(color = "black"),  # Black title for x-axis
    axis.title.y = element_text(color = "black")   # Black title for y-axis
  ) +
  scale_y_continuous(labels = scales::percent)  # Show y-axis in percentage

## Registering Organization

In [ ]:
# Replace NA values with the string "NA" and summarize the data
pivot_table_registering_org <- nursery_survey_data %>%
  mutate(registering_org = ifelse(is.na(registering_org), "NA", registering_org)) %>%  # Replace NA with "NA"
  group_by(ADM1, registering_org) %>%
  summarise(count = n(), .groups = "drop")

# Ensure no empty slices (filter any counties with zero counts)
pivot_table_registering_org <- pivot_table_registering_org %>%
  filter(count > 0)

# Create a pie chart
ggplot(pivot_table_registering_org, aes(x = "", y = count, fill = registering_org)) +
  geom_bar(stat = "identity", width = 1) +  # Create the bar for the pie
  coord_polar(theta = "y") +  # Convert the bar chart to a pie chart
  facet_wrap(~ ADM1, scales = "free") +  # Facet by county (ADM1) with free scales
  labs(
    title = "Distribution of Registering Organizations by County",
    fill = "Registering Organization"
  ) +
  theme_void() +  # Clean up the chart for a more aesthetic look
  theme(axis.text.x = element_blank())  # Remove x-axis text

Nakuru stands out with a significantly higher number of registered organizations compared to the others, indicating a stronger presence of registered nurseries or a more active registration process in this region

# Analysing the Nursery Production Category per county
## Production Category Per County

In [ ]:
# Create the pivot table: Count per nursery production category
pivot_table_nursery_production <- nursery_survey_data %>%
  group_by(ADM1, nursery_production_category) %>%
  count() %>%  # Count the occurrences of each category per county
  spread(key = nursery_production_category, value = n, fill = 0)  # Reshape to wide format

# View the pivot table
print(pivot_table_nursery_production)

## Number of Seedlings available Per County

In [ ]:
# Create a pivot table for the total number of seedlings per county and arrange in descending order
seedlings_count_per_county <- nursery_survey_data %>%
  group_by(ADM1) %>%  # Group by county
  summarize(total_seedlings = sum(number_of_seedlings, na.rm = TRUE), .groups = "drop") %>%  # Sum of seedlings per county
  arrange(desc(total_seedlings))  # Arrange in descending order

# View the pivot table
print(seedlings_count_per_county)

## Method Of Propagation

In [ ]:
pivot_method_of_propagation <- nursery_survey_data %>%
  group_by(ADM1, method_of_propagation) %>%
  summarise(
    count = n(),  # Count the occurrences of each method per county
    .groups = "drop"
  ) %>%
  pivot_wider(
    names_from = method_of_propagation,  # Spread methods as columns
    values_from = count,                # Values for each method
    values_fill = list(count = 0)       # Replace missing values with 0
  ) %>%
  mutate(
    total = `wildings` + `wildings and seeds` + `seeds` + `vegetative`  # Add a total column
  )

# View the resulting pivot table
print(pivot_method_of_propagation)

# Analysis of the Distance to Paved Road and Water
## Distance of the Nurseris to the Paved Road

In [ ]:
max_distance_per_county <- nursery_survey_data %>%
  group_by(ADM1) %>%
  summarise(max_distance = max(distance_to_paved_road, na.rm = TRUE), .groups = 'drop')

# Create the bar chart with the maximum distance and labels on top of each bar
ggplot(max_distance_per_county, aes(x = reorder(ADM1, max_distance),
                                    y = max_distance, fill = ADM1)) +
  geom_bar(stat = "identity", position = "dodge") +  # Bar chart with max distance per county
  labs(
    title = "Maximum Distance to Paved Road by County",
    x = "County",
    y = "Maximum Distance to Paved Road (in km)",
    fill = "County"
  ) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1, color="black")) +  # Rotate x-axis labels for readability
  scale_fill_brewer(palette = "Set3") +  # Use a readable color palette for fill
  scale_y_continuous(labels = function(x) paste0(x, " km")) +  # Add "km" suffix to y-axis labels
  geom_text(aes(label = paste0(max_distance, " km")),  # Display the max distance on top of each bar
            vjust = -0.5,  # Adjust vertical position (above the bar)
            size = 3,  # Adjust text size
            color = "black")  # Text color

## Distance To Water

In [ ]:
max_distance_to_water_per_county <- nursery_survey_data %>%
  group_by(ADM1) %>%
  summarise(max_distance_to_water = max(distance_to_water, na.rm = TRUE), .groups = 'drop')

# Create the bar chart with the maximum distance to water and labels on top of each bar
ggplot(max_distance_to_water_per_county, aes(x = reorder(ADM1, max_distance_to_water),
                                              y = max_distance_to_water, fill = ADM1)) +
  geom_bar(stat = "identity", position = "dodge") +  # Bar chart with max distance to water per county
  labs(
    title = "Maximum Distance to Water by County",
    x = "County",
    y = "Maximum Distance to Water (in km)",
    fill = "County"
  ) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1, color ="black")) +  # Rotate x-axis labels for readability
  scale_fill_brewer(palette = "Set3") +  # Use a readable color palette for fill
  scale_y_continuous(labels = function(x) paste0(x, " km")) +  # Add "km" suffix to y-axis labels
  geom_text(aes(label = paste0(max_distance_to_water, " km")),  # Display the max distance to water on top of each bar
            vjust = -0.5,  # Adjust vertical position (above the bar)
            size = 3,  # Adjust text size
            color = "black")  # Text color

# ANALYSIS FOR EMPLOYEES
## Identifying the total number of employeers Per County

In [ ]:
pivot_table <- nursery_survey_data %>%
  group_by(ADM1) %>%
  summarise(
    total_nursery_employees = sum(nursery_total_employees, na.rm = TRUE),
    total_male_employees = sum(males_total, na.rm = TRUE),
    total_female_employees = sum(females_total, na.rm = TRUE),
    mean_male_percentage = round((total_male_employees / total_nursery_employees) * 100, 2),  # Mean of male percentage
    mean_female_percentage = round((total_female_employees / total_nursery_employees) * 100, 2),  # Mean of female percentage
    .groups = 'drop'
  )

# View the result
print(pivot_table)

# Checking for Counties that Practise Nursery Technques for the different species

In [ ]:
# Create a pivot table for Nursery_Techniques per county (ADM1)
pivot_table_nursery_techniques <- nursery_survey_data %>%
  filter(!is.na(Nursery_Techniques)) %>%  # Remove NAs from Nursery_Techniques column
  group_by(ADM1, Nursery_Techniques) %>%  # Group by County and Nursery Techniques (YES/NO)
  summarize(Count = n(), .groups = "drop") %>%  # Count occurrences
  pivot_wider(names_from = Nursery_Techniques, values_from = Count, values_fill = list(Count = 0))  # Pivot the table

# View the pivot table
print(pivot_table_nursery_techniques)

It is evident that Kjiado County do not practice nursery techniques for the different species and also do not take part in the seed collection. All the rest take part in Nursery techniques apart from some but for some from 16 from Nakuru and 5 from Nyandarua.

Those who are not sales argue that they do not take part in Seed Sales because they cannot meet demands or are yet to make sales, they don't sale since their goal is restoration and conservation, They have partnered with KFS to plant in their forests and they support them in marketing the seedlings,they have the reliable project market from the ITF or their in-house market demand is very promising

# SPECIES ANALYSIS
# ANALYSING SELLERS
## Checking whether the sellers attended training offered

In [ ]:
# Create the pivot table for training_attended per county with "No" changed to "no"
pivot_table_training_attended <- nursery_survey_data %>%
  mutate(training_attended = tolower(training_attended)) %>%  # Convert "No" to "no"
  group_by(ADM1, training_attended) %>%
  summarize(count = n(), .groups = "drop") %>%
  pivot_wider(names_from = training_attended, values_from = count, values_fill = list(count = 0))

# View the pivot table
View(pivot_table_training_attended)

## Analysis from the sellers whether the training was helpful for them in relation to the theme

In [ ]:
training_theme_table <- nursery_survey_data %>%
  group_by(training_theme, ADM1, training_helpful) %>%  # Group by training theme, county (ADM1), and helpfulness
  summarize(count = n(), .groups = "drop")  # Count the occurrences without pivoting

# View the result
View(training_theme_table)


From the table we can see that 35 themes were involved the training sessions, 21 received no training there were NA's from the data showing the training sessions were not availed as expected in all counties. Only 2 organizations provided the Training, that is, ICRAF and Green Belt Movement

# MARKET ANALYSIS
## Established Markets Per County

In [ ]:
# Summarize data for Established markets per county
established_markets_summary <- nursery_survey_data %>%
  group_by(ADM1, Established_Market) %>%
  summarize(count = n(), .groups = "drop")

# Create a stacked bar chart and add counts on top of the bars
ggplot(established_markets_summary, aes(x = ADM1, y = count, fill = Established_Market)) +
  geom_bar(stat = "identity", position = "stack") +  # Stacked bar chart
  geom_text(aes(label = count), position = position_stack(vjust = 0.5), size = 3.5) +  # Add count labels inside the bars
  labs(
    title = "Established Markets Per County",
    x = "County",
    y = "Count of Established Markets",
    fill = "Established Markets"
  ) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1, colour="black"))

## Identifying the main customers from Each of the Counties

In [ ]:
# Step 1: Clean up and standardize the 'Main_Customers' column
nursery_survey_data <- nursery_survey_data %>%
  mutate(Main_Customers = case_when(
    Main_Customers == "private_individuals in_house_needs" ~ "in_house_needs private_individuals",
    Main_Customers == "in_house_needs NGOs companies" ~ "in_house_needs NGOs and companies",
    Main_Customers == "NGOs government_planting_projects companies" ~ "NGOs and government_planting_projects companies",
    Main_Customers == "in_house_needs and companies" ~ "in_house_needs and companies",
    Main_Customers == "in_house_needs government_planting_projects" ~ "in_house_needs and government_planting_projects",
    Main_Customers == "private_individuals companies others" ~ "private_individuals and companies others",
    TRUE ~ Main_Customers  # Keep all other values unchanged
  ))

# Step 2: Filter the relevant categories and summarize the counts and percentages by county
main_customers_summary <- nursery_survey_data %>%
  filter(Main_Customers %in% c("government_planting_projects",
                               "private_individuals",
                               "in_house_needs",
                               "NGOs",
                               "in_house_needs private_individuals",
                               "in_house_needs NGOs and companies",
                               "NGOs and government_planting_projects companies",
                               "in_house_needs and companies",
                               "in_house_needs and government_planting_projects",
                               "private_individuals and companies others")) %>%
  group_by(ADM1, Main_Customers) %>%
  summarize(count = n(), .groups = "drop") %>%
  group_by(ADM1) %>%
  mutate(percentage = round(count / sum(count) * 100, 2))  # Calculate percentage per county and round to 2 decimals

# Step 3: Pivot the data so that counties are columns and Main_Customers are rows
pivot_table <- main_customers_summary %>%
  select(ADM1, Main_Customers, percentage) %>%
  spread(key = ADM1, value = percentage, fill = 0)  # Pivoting, filling missing values with 0

# Display the pivot table with rounded percentages
pivot_table

## Species Preferred Per County

In [ ]:
# Calculate the count and percentage of preferred species per county
species_preferred_summary <- nursery_survey_data %>%
  group_by(ADM1, Species_Preferred) %>%
  summarize(count = n(), .groups = "drop") %>%
  group_by(ADM1) %>%
  mutate(percentage = round(count / sum(count) * 100, 2))  # Calculate percentage within each county

# Pivot the data so that counties are columns and species preferred are rows
pivot_table <- species_preferred_summary %>%
  select(ADM1, Species_Preferred, percentage) %>%
  spread(key = ADM1, value = percentage, fill = 0)  # Pivoting, filling missing values with 0

# Display the pivot table
View(pivot_table)

## Marketing Medium

In [ ]:
marketing_medium_summary <- nursery_survey_data %>%
  group_by(ADM1, Marketing_Medium) %>%
  summarize(count = n(), .groups = "drop") %>%
  group_by(ADM1) %>%
  mutate(percentage = round(count / sum(count) * 100, 2))  # Calculate and round percentage to 2 decimal places

# Pivot the data to display counties as columns and marketing mediums as rows
pivot_table <- marketing_medium_summary %>%
  spread(key = ADM1, value = percentage, fill = 0)  # Pivot counties as columns, showing percentages

# Print the pivot table
View(pivot_table)

## Analysis of The Type of Constraints in the Market Place

In [ ]:
# Summarize the count of Constraints_To_Reaching_Customers per county
constraints_summary <- nursery_survey_data %>%
  group_by(ADM1, Constraints_To_Reaching_Customers) %>%
  summarize(count = n(), .groups = "drop")

# Calculate the percentage for each constraint per county
constraints_summary <- constraints_summary %>%
  group_by(ADM1) %>%
  mutate(percentage = count / sum(count) * 100)

# Plot pie charts for each county
ggplot(constraints_summary, aes(x = "", y = percentage, fill = Constraints_To_Reaching_Customers)) +
  geom_bar(stat = "identity", width = 1) +  # Create the bar chart
  coord_polar(theta = "y") +  # Convert to pie chart
  facet_wrap(~ ADM1) +  # Create a separate pie chart for each county
  labs(
    title = "Distribution of Constraints to Reaching Customers per County",
    fill = "Constraints"
  ) +
  theme_minimal() +
  theme(axis.text.x = element_blank(),  # Hide x-axis text
        axis.ticks = element_blank(),  # Remove axis ticks
        panel.grid = element_blank())  # Remove grid lines

## Market Information Source

In [ ]:
mgt_info_summary <- nursery_survey_data %>%
  group_by(ADM1, nursery_mgt_information_source) %>%
  summarize(count = n(), .groups = "drop")

# Pivot the data to a wide format to display actual counts per county
pivot_table <- mgt_info_summary %>%
  spread(key = nursery_mgt_information_source, value = count, fill = 0)  # Pivoting the data

# Display the pivot table without kable
View(pivot_table)

# NURSERY MANAGEMENT ANALYSIS
## Checking if the Nurseries in the particular counties plan on scaling up

In [ ]:
# Summarize the count of nursery_mgt_information_source per county
mgt_info_summary <- nursery_survey_data %>%
  group_by(ADM1, nursery_mgt_information_source) %>%
  summarize(count = n(), .groups = "drop")

# Calculate the total count per county
county_total <- mgt_info_summary %>%
  group_by(ADM1) %>%
  summarize(total_count = sum(count), .groups = "drop")

# Join the total count back to the original summary table to calculate percentages
mgt_info_summary_with_percentage <- mgt_info_summary %>%
  left_join(county_total, by = "ADM1") %>%
  mutate(percentage = round((count / total_count) * 100, 2))

# Select and display the final table
final_table <- mgt_info_summary_with_percentage %>%
  select(ADM1, nursery_mgt_information_source, count, total_count, percentage) %>%
  rename(
    county = ADM1,
    total = total_count
  )

# Print the final table
View(final_table)

## Competitors per County

In [ ]:
# Summarize competitors_list by county
competitors_summary <- nursery_survey_data %>%
  group_by(ADM1, competitors_list) %>%
  summarize(count = n(), .groups = "drop") %>%
  spread(key = competitors_list, value = count, fill = 0)  # Pivot table format

# View the pivot table
View(competitors_summary)

# Training Areas Required

In [ ]:
# Summarize the data by county and training areas required
training_summary <- nursery_survey_data %>%
  group_by(ADM1, training_areas_required) %>%
  summarize(count = n(), .groups = "drop")

# Pivot the table to make counties the columns and training areas as rows
pivot_table <- training_summary %>%
  spread(key = ADM1, value = count, fill = 0)  # Pivot counties as columns

# Display the pivot table
View(pivot_table)